# SPARC AMP Example

This notebook contains a complete worked example of a SPARC AMP encoder/decoder using Hadamard design matrices and various power allocation strategies.

For full details of the algorithm please see our paper _"Capacity-achieving Sparse Superposition Codes via Approximate Message Passing Decoding_": https://arxiv.org/abs/1501.05892

For further optimisation details please see the follow up paper _"Optimising the Finite-Length Performance of AMP-decoded Sparse Regression Codes"_.

This notebook is Copyright 2016, 2017 Adam Greig.

## Requirements

We need numpy and ideally an accelerated Hadamard transform written in C, available to install as the `pyfht` library (try `sudo pip install pyfht` or similar for your Python package installation system).

If `pyfht` is not available we can fall back to a Python version, but beware it is a great deal slower.

In [1]:
import numpy as np

try:
    from pyfht import fht_inplace
except ImportError:
    import warnings
    warnings.warn("Using very slow Python version of fht, please install pyfht")
    def fht_inplace(x):
        N = len(x)
        i = N>>1
        while i:
            for k in range(0, N, 2*i):
                for j in range(k, i+k):
                    ij = i|j
                    temp = x[j]
                    x[j] += x[ij]
                    x[ij] = temp - x[ij]
            i = i >> 1

## Fast Hadamard Transforms

These functions use the in-place Hadamard transform to emulate matrix operations on a random $\pm1$ matrix of any size smaller than the underlying Hadamard matrix. We then also combine $L$ such operations for efficiency when using very wide matrices, typical for SPARC dictionaries.

None of this code is particularly relevant to the actual SPARC-AMP decoder, so feel free to skip over it.

In [2]:
def sub_fht(n, m, seed=0, ordering=None):
    """
    Returns functions to compute the sub-sampled Walsh-Hadamard transform,
    i.e., operating with a wide rectangular matrix of random +/-1 entries.

    n: number of rows
    m: number of columns

    It is most efficient (but not required) for max(m+1,n+1) to be a power of 2.

    seed: determines choice of random matrix
    ordering: optional n-long array of row indices in [1, max(m,n)] to
              implement subsampling; generated by seed if not specified,
              but may be given to speed up subsequent runs on the same matrix.

    Returns (Ax, Ay, ordering):
        Ax(x): computes A.x (of length n), with x having length m
        Ay(y): computes A'.y (of length m), with y having length n
        ordering: the ordering in use, which may have been generated from seed
    """
    assert n > 0, "n must be positive"
    assert m > 0, "m must be positive"
    w = 2**int(np.ceil(np.log2(max(m+1, n+1))))

    if ordering is not None:
        assert ordering.shape == (n,)
    else:
        rng = np.random.RandomState(seed)
        idxs = np.arange(1, w, dtype=np.uint32)
        rng.shuffle(idxs)
        ordering = idxs[:n]

    def Ax(x):
        assert x.size == m, "x must be m long"
        y = np.zeros(w)
        y[w-m:] = x.reshape(m)
        fht_inplace(y)
        return y[ordering]

    def Ay(y):
        assert y.size == n, "input must be n long"
        x = np.zeros(w)
        x[ordering] = y
        fht_inplace(x)
        return x[w-m:]

    return Ax, Ay, ordering

def block_sub_fht(n, m, l, seed=0, ordering=None):
    """
    As `sub_fht`, but computes in `l` blocks of size `n` by `m`, potentially
    offering substantial speed improvements.

    n: number of rows
    m: number of columns per block
    l: number of blocks

    It is most efficient (though not required) when max(m+1,n+1) is a power of 2.

    seed: determines choice of random matrix
    ordering: optional (l, n) shaped array of row indices in [1, max(m, n)] to
              implement subsampling; generated by seed if not specified, but
              may be given to speed up subsequent runs on the same matrix.

    Returns (Ax, Ay, ordering):
        Ax(x): computes A.x (of length n), with x having length l*m
        Ay(y): computes A'.y (of length l*m), with y having length n
        ordering: the ordering in use, which may have been generated from seed
    """
    assert n > 0, "n must be positive"
    assert m > 0, "m must be positive"
    assert l > 0, "l must be positive"

    if ordering is not None:
        assert ordering.shape == (l, n)
    else:
        w = 2**int(np.ceil(np.log2(max(m+1, n+1))))
        rng = np.random.RandomState(seed)
        ordering = np.empty((l, n), dtype=np.uint32)
        idxs = np.arange(1, w, dtype=np.uint32)
        for ll in range(l):
            rng.shuffle(idxs)
            ordering[ll] = idxs[:n]

    def Ax(x):
        assert x.size == l*m
        out = np.zeros(n)
        for ll in range(l):
            ax, ay, _ = sub_fht(n, m, ordering=ordering[ll])
            out += ax(x[ll*m:(ll+1)*m])
        return out

    def Ay(y):
        assert y.size == n
        out = np.empty(l*m)
        for ll in range(l):
            ax, ay, _ = sub_fht(n, m, ordering=ordering[ll])
            out[ll*m:(ll+1)*m] = ay(y)
        return out

    return Ax, Ay, ordering

## SPARC Dictionary

We use the `block_sub_fht` which computes the equivalent of $A\beta$ by using $L$ separate $M\times M$ Hadamard matrices. However we want each entry to be divided by $\sqrt{n}$ to get the right variance, and we need to do a reshape on the output to get column vectors, so we'll wrap those operations here.

Returns two functions `Ab` and `Az` which compute $A\beta$ and $A^Tz$ respectively.

In [3]:
def sparc_transforms(L, M, n):
    Ax, Ay, _ = block_sub_fht(n, M, L, ordering=None)
    def Ab(b):
        return Ax(b).reshape(-1, 1) / np.sqrt(n)
    def Az(z):
        return Ay(z).reshape(-1, 1) / np.sqrt(n)
    return Ab, Az

## Power Allocation

There are multiple choices for power allocation algorithm. Here we give the "standard exponential" allocation used for the capacity-achieving proof; a similar exponential with parameterised steepness which is flattened after some point; and a newer iterative allocation which aims to allocate just sufficient power for blocks of sections to decode successively. You can select which is used by altering the function called from `amp_sim` below.

### Original
This power allocation is:
$P_\ell \propto 2^{-2\mathcal{C}\ell/L}$

In [4]:
def pa_original(L, C, P):
    pa = 2**(-2 * a * C * np.arange(L) / L)
    pa /= pa.sum() / P
    return pa

### Parameterised

We generate a power allocation parameterised by $a$ and $f$ which set the steepness and the flattening point respectively.

The power allocation follows $P_\ell \propto 2^{-2a\mathcal{C}\ell/L}$ up to $fL$, and then is flat after that point, and scaled so that the sum power is $P$.

In [5]:
def pa_parameterised(L, C, P, a, f):
    pa = 2**(-2 * a * C * np.arange(L) / L)
    pa[int(f*L):] = pa[int(f*L)]
    pa /= pa.sum() / P
    return pa

### Iterative

We split the sections into $B$ blocks of $L/B$ sections each, where each section in a block has the same power. Then each block is assigned just enough power to meet the requirement $P_\ell > 2(\ln 2)R\tau_t^2 / L$. In practice the rate $R_\text{PA}$ used here is set independently from the actual codeword rate to allow altering the power allocation. When the total unallocated power is sufficient to give all remaining sections the same power, we do that to complete the allocation.

One modification is to pass $B=L$, so each block contains only one section. This tends to perform better when given sufficient iterations.

For further details on this power allocation, see _"Optimising the Finite-Length Performance of AMP-decoded Sparse Regression Codes"_.

In [6]:
def pa_iterative(L, B, σ, P, R_PA):
    PA = np.zeros(L)
    τ = np.zeros(B)
    k = L//B
    for b in range(B):
        Premain = P - PA.sum()
        τ[b] = np.sqrt(σ**2 + Premain)
        Pblock = 2 * np.log(2) * (R_PA/L) * τ[b]**2
        Pspread = Premain / (L - k*b)
        if Pblock > Pspread:
            PA[k*b:k*(b+1)] = Pblock
        else:
            PA[k*b:] = Pspread
            break
    return PA

## AMP
This is the actual AMP algorithm. It's a mostly straightforward transcription from the relevant equations, but we regularise the exponential terms to prevent overflow and improve numerical precision.

We estimate $\tau^2$ from $\mathrm{Var}[z]$ rather than pre-computed constants.
If $\tau^2$ hasn't changed between two iterations, we can stop early.
For further details on these optimisations, see _"Optimising the Finite-Length Performance of AMP-decoded Sparse Regression Codes"_.

In [7]:
def amp(y, σ_n, Pl, L, M, T, Ab, Az):
    P = np.sum(Pl)
    n = y.size
    β = np.zeros((L*M, 1))
    z = y
    last_τ = 0
    
    for t in range(T):
        τ = np.sqrt(np.sum(z**2)/n)
        if τ == last_τ:
            return β
        last_τ = τ
        
        s = β + Az(z)
        rt_n_Pl = np.sqrt(n*Pl).repeat(M).reshape(-1, 1)
        u = s * rt_n_Pl / τ**2
        max_u = u.max()
        exps = np.exp(u - max_u)
        sums = exps.reshape(L, M).sum(axis=1).repeat(M).reshape(-1, 1)
        β = (rt_n_Pl * exps / sums).reshape(-1, 1)
        z = y - Ab(β) + (z/τ**2) * (P - np.sum(β**2) / n)
    
    return β

## SPARC-AMP Simulation
Here we perform the actual simulation. The comments inline guide you through what's going on.

In [8]:
def amp_sim(L, M, σ_n, P, R, T, R_PA):
    # Compute the SNR, capacity, and n, from the input parameters
    snr = P / σ_n**2
    C = 0.5 * np.log2(1 + snr)
    n = int(L*np.log2(M) / R)
    
    # Generate the power allocation
    Pl = pa_iterative(L, L, σ_n, P, R_PA)
    
    # Generate random message in [0..M)^L
    tx_message = np.random.randint(0, M, L).tolist()
    
    # Generate the SPARC transform functions A.beta and A'.z
    Ab, Az = sparc_transforms(L, M, n)
    
    # Generate our transmitted signal X
    β_0 = np.zeros((L*M, 1))
    for l in range(L):
        β_0[l*M + tx_message[l]] = np.sqrt(n * Pl[l])
    x = Ab(β_0)
    
    # Generate random channel noise and then received signal y
    z = np.random.randn(n, 1) * σ_n
    y = (x + z).reshape(-1, 1)
        
    # Run AMP decoding
    β = amp(y, σ_n, Pl, L, M, T, Ab, Az).reshape(-1)
    
    # Convert decoded beta back to a message
    rx_message = []
    for l in range(L):
        idx = np.argmax(β[l*M:(l+1)*M])
        rx_message.append(idx)
    
    # Compute fraction of sections decoded correctly
    correct = np.sum(np.array(rx_message) == np.array(tx_message)) / L
    
    # Compute BER
    ber = np.sum(bin(a^b).count('1')
                 for (a, b) in zip(tx_message, rx_message))/(L*np.log2(M))
    
    # Compute Eb/N0
    EbN0 = 1/(2*R) * (P/σ_n**2)
    
    return {
        "L": L, "M": M, "sigma_n": σ_n, "P": P, "R": R, "T": T,
        "snr": snr, "C": C, "n": n, "fc": correct, "R_PA": R_PA,
        "ber": ber, "EbN0": EbN0, "ser": 1-correct
    }

Let's have a trial run.

In [9]:
np.random.seed(0)
amp_sim(1024, 512, 1.0, 15.0, 1.4, 64, 1.4)

{'C': 2.0,
 'EbN0': 5.357142857142858,
 'L': 1024,
 'M': 512,
 'P': 15.0,
 'R': 1.4,
 'R_PA': 1.4,
 'T': 64,
 'ber': 0.0,
 'fc': 1.0,
 'n': 6582,
 'ser': 0.0,
 'sigma_n': 1.0,
 'snr': 15.0}

`fc` is the fraction of sections decoded correctly - 100% in this case, so no errors.